In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 60.9 MB/s eta 0:00:00


In [ ]:
import json
import random
from scipy.spatial import distance_matrix
import numpy as np
import pulp

# Function to create nodes with random coordinates
def generate_nodes(num_nodes=10):
    """
    Generates nodes with random coordinates (x, y).
    Ensures exactly `num_nodes` nodes are created.
    """
    nodes = {i: {"x": random.uniform(0, 100), "y": random.uniform(0, 100)} for i in range(num_nodes)}
    return nodes

def generate_edges_with_features(nodes):
    """
    Generates edges with features as integers between 1 and 10.
    Features are:
    - feature1: Random integer between 1 and 10
    - feature2: Random integer between 1 and 10
    - feature3: Random integer between 1 and 10
    - feature4: Random integer between 1 and 10
    - feature5: Random integer between 1 and 10
    """
    edges = []
    num_nodes = len(nodes)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                edge = {
                    "from": i,
                    "to": j,
                    "feature1": random.randint(1, 10),  # Random integer in range [1, 10]
                    "feature2": random.randint(1, 10),  # Random integer in range [1, 10]
                    "feature3": random.randint(1, 10),  # Random integer in range [1, 10]
                    "feature4": random.randint(1, 10),  # Random integer in range [1, 10]
                    "feature5": random.randint(1, 10),  # Random integer in range [1, 10]
                }
                edges.append(edge)

    return edges




# Function to calculate the cost of an edge
def calculate_edge_cost(edge, weights):
    """
    Calculate the cost of an edge based on its features and weights.
    Adds a small random error term (ϵ) to simulate real-world uncertainty.
    """
    base_cost = (weights[0] * edge["feature1"] +
                 weights[1] * edge["feature2"] +
                 weights[2] * edge["feature3"] +
                 weights[3] * edge["feature4"] +
                 weights[4] * edge["feature5"])
    error_term = random.uniform(-0.1, 0.1)  # Example: small random error in the range [-0.1, 0.1]
    return base_cost + error_term

# TSP solver function using PuLP
def solve_tsp_with_pulp(nodes, edges, weights):
    num_nodes = len(nodes)
    model = pulp.LpProblem("TSP", pulp.LpMinimize)
    x = {(i, j): pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for i in range(num_nodes) for j in range(num_nodes) if i != j}

    # Creating a cost matrix
    cost = {}
    for edge in edges:
        i, j = edge["from"], edge["to"]
        cost_value = calculate_edge_cost(edge, weights)
        cost[(i, j)] = cost_value

    # Objective: Minimize total cost
    model += pulp.lpSum(cost[i, j] * x[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j)

    # Constraints: Each node has one incoming and one outgoing path
    for k in range(num_nodes):
        model += pulp.lpSum(x[i, k] for i in range(num_nodes) if i != k) == 1
        model += pulp.lpSum(x[k, j] for j in range(num_nodes) if j != k) == 1

    # Subtour elimination (MTZ constraints)
    u = {i: pulp.LpVariable(f"u_{i}", lowBound=0, upBound=num_nodes - 1, cat="Continuous") for i in range(1, num_nodes)}
    for i in range(1, num_nodes):
        for j in range(1, num_nodes):
            if i != j:
                model += u[i] - u[j] + num_nodes * x[i, j] <= num_nodes - 1

    # Solve the optimization problem
    model.solve()

    # Extract the optimal tour
    tour = []
    if pulp.LpStatus[model.status] == "Optimal":
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j and pulp.value(x[i, j]) == 1:
                    tour.append((i, j))

    return tour

# Function to calculate the total cost of the optimal tour
def calculate_total_cost(optimal_tour, edges, weights):
    edge_costs = {(edge["from"], edge["to"]): calculate_edge_cost(edge, weights) for edge in edges}
    total_cost = sum(edge_costs[edge] for edge in optimal_tour)
    return total_cost

# Main function to create a TSP graph with optimal solution
# Main function to create a TSP graph with optimal solution and cost matrix
def create_tsp_graph_with_optimal_solution(weights):
    nodes = generate_nodes(10)  # Always 10 nodes
    edges = generate_edges_with_features(nodes)

    # Compute the optimal tour
    optimal_tour = solve_tsp_with_pulp(nodes, edges, weights)

    # Calculate total cost of the optimal tour
    total_cost = calculate_total_cost(optimal_tour, edges, weights)

    # Compute the cost matrix
    num_nodes = len(nodes)
    cost_matrix = np.full((num_nodes, num_nodes), float('inf'))  # Initialize with infinity for non-existing edges
    for edge in edges:
        i, j = edge["from"], edge["to"]
        cost_matrix[i, j] = calculate_edge_cost(edge, weights)

    # Store graph data in a dictionary
    graph_data = {
        "nodes": nodes,
        "edges": edges,
        "optimal_tour": optimal_tour,
        "total_cost": total_cost,
        "cost_matrix": cost_matrix.tolist()  # Convert to list for JSON serialization
    }
    return graph_data


# Function to create and save multiple graphs with optimal solutions
def save_graph_dataset_with_optimal_solutions(num_graphs, filename="tsp_dataset_with_optimal50+error0_1.json"):
    weights = [0.8, 0.3, 0.4, 0.5, 0.6]  # Example weights for features
    dataset = [create_tsp_graph_with_optimal_solution(weights) for _ in range(num_graphs)]

    with open(filename, 'w') as f:
        json.dump(dataset, f, indent=4)

# Example call to create the dataset
save_graph_dataset_with_optimal_solutions(50, "tsp_dataset_with_optimal50+error0_1.json")
